In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

2024-02-04 20:45:19.273470: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
import os
import cv2
import numpy as np

# Function to extract temperature from an RGB color
def extract_temperature_from_color(color):
    # This is a simplified example, you would need to create a mapping based on your color scale
    # Example: linear mapping between RGB values and temperature (adjust based on your scale)
    temperature = 10.0 * color[0] + 20.0 * color[1] + 30.0 * color[2]
    return temperature

# Function to load and preprocess data
def load_and_preprocess_data(folder_path, image_size, roi_coordinates):
    # Initialize empty lists to store images and labels
    images = []
    labels = []

    # Iterate through the files in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg"):  # Assuming images are in JPG format
            file_path = os.path.join(folder_path, filename)

            # Read and resize the image
            img = cv2.imread(file_path)
            img = cv2.resize(img, (image_size, image_size))

            # Extract temperature values from the color-coded ROI
            roi = img[roi_coordinates[0]:roi_coordinates[1], roi_coordinates[2]:roi_coordinates[3]]
            temperatures = np.zeros_like(roi, dtype=float)
            for i in range(roi.shape[0]):
                for j in range(roi.shape[1]):
                    temperatures[i, j] = extract_temperature_from_color(roi[i, j])

            # Append the image and average temperature to the lists
            images.append(img)
            # Use the average temperature of the ROI as the label
            average_temperature = np.mean(temperatures)
            labels.append(average_temperature)

    # Convert lists to NumPy arrays
    images = np.array(images)
    labels = np.array(labels)

    # Normalize image data to values between 0 and 1
    images = images / 255.0

    return images, labels

# Set the path to your folder containing images
folder_path = "/Users/samagra/INCOIS/SST SAT Images"

# Set the desired image size (adjust as needed based on your model architecture)
image_size = 128

# Set the coordinates of the ROI in the format (top_left_y, bottom_right_y, top_left_x, bottom_right_x)
roi_coordinates = (26, 63, 15, 808)  # Adjust these coordinates based on the actual location of the color-coded region

# Load and preprocess the data
X, y = load_and_preprocess_data(folder_path, image_size, roi_coordinates)

# Now, X contains the preprocessed images, and y contains the corresponding average temperature labels


In [15]:
image_height = image_width = image_size
num_channels = 3  # Assuming RGB images, adjust if your images have a different number of channels

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_height, image_width, num_channels)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Output layer for regression task
])

model.compile(optimizer='adam', loss='mean_squared_error')  # Adjust optimizer and loss function as needed

In [16]:
# Filter out data points with None temperatures
valid_indices = np.where(y_train != None)

if len(valid_indices[0]) > 0:
    X_train_filtered = X_train[valid_indices]
    y_train_filtered = y_train[valid_indices]

    # Model training with reduced validation split
    model.fit(X_train_filtered, y_train_filtered, epochs=10, validation_split=0.1)
else:
    print("No valid samples remaining after filtering.")


No valid samples remaining after filtering.


In [17]:
# Load and preprocess your test data (replace with your data loading logic)
# X_test, y_test = load_and_preprocess_test_data()

# Normalize test image data
X_test = X_test / 255.0

# Make predictions
predictions = model.predict(X_test)

1/1 [==============================] - 1s 1s/step


In [ ]:
# Choose an example image and its corresponding prediction
index = 0
predicted_temperature = predictions[index][0]

# Visualize the image
plt.imshow(X_test[index])
plt.title(f'Predicted Temperature: {predicted_temperature}')
plt.colorbar()
plt.show()